In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../datasets/KOPIS/performance.csv')

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_5268\3718354033.py:1: DtypeWarning: Columns (30,32,33,35,36,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../datasets/KOPIS/performance.csv')


In [4]:
df.shape

(35630, 40)

In [5]:
df.columns

Index(['공연ID', '공연명', '공연시작일', '공연종료일', '공연시설명', '출연진', '제작진', '런타임', '관람연령',
       '제작사', '기획사', '주최', '주관', '티켓가격', '포스터URL', '줄거리', '공연지역', '공연장르명',
       '오픈런', '내한여부', '아동공연여부', '대학로공연여부', '축제여부', '뮤지컬라이센스', '뮤지컬창작', '최종수정일',
       '공연상태', '공연시설ID', '공연시간', '소개이미지목록', '장르(통계)', '상연횟수', '공연명(통계)',
       '공연시설명(통계)', '제작/기획사(통계)', '공연시작일(통계)', '공연종료일(통계)', '연도(통계)', '월(통계)',
       '장르코드(통계)'],
      dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35630 entries, 0 to 35629
Data columns (total 40 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   공연ID        35630 non-null  object 
 1   공연명         35628 non-null  object 
 2   공연시작일       35628 non-null  object 
 3   공연종료일       35628 non-null  object 
 4   공연시설명       35612 non-null  object 
 5   출연진         22837 non-null  object 
 6   제작진         15795 non-null  object 
 7   런타임         34072 non-null  object 
 8   관람연령        34912 non-null  object 
 9   제작사         15546 non-null  object 
 10  기획사         4161 non-null   object 
 11  주최          21919 non-null  object 
 12  주관          15439 non-null  object 
 13  티켓가격        34555 non-null  object 
 14  포스터URL      34924 non-null  object 
 15  줄거리         2623 non-null   object 
 16  공연지역        35627 non-null  object 
 17  공연장르명       35628 non-null  object 
 18  오픈런         35628 non-null  object 
 19  내한여부        35628 non-nul

In [ ]:
# 결측치 삭제
df.drop(columns=['소개이미지목록', '제작/기획사(통계)'], axis=1, inplace=True)

In [20]:
# 티켓가격 탐색
df['티켓가격'].value_counts().tail(20)

티켓가격
1인석 77,000원, 2인석 154,000원                                                                                         1
R석 (2부작) 66,000원, S석 (2부작) 44,000원, R석 (3부작) 88,000원, S석 (3부작) 55,000원                                            1
사전예약 30,000원, 현장판매 35,000원, 오프라인 티켓＋포스터 40,000원                                                                   1
전석_1회차 공연 50,000원, 전석_2회차 공연 55,000원                                                                              1
R석 121,000원, S석 110,000원, A석 88,000원, B석 77,000원                                                                  1
C석 44,000원                                                                                                        1
오프라인 티켓 22,000원, 온라인 티켓 5,000원, 현장판매 25,000원                                                                      1
전석 55,000원, 전석 66,000원                                                                                            1
R석 120,000원, S석 99,000원, A석 77,000원                                

In [ ]:
def clean_tickek_data(df):
    # 1) 디너쇼 제거
    mask_dinnershow = df['공연명'].str.contains('디너쇼|디너 쇼|디너콘서트|디너 콘서트', na=False)
    
    # 2) 멀티데이권 제거
    mask_multiday = df['티켓가격'].str.contains('양일권|2일권|3일권|전일권', na=False)
    
    # 3) 패키지 제거
    mask_package = df['티켓가격'].str.contains('패키지|페키지', na=False)
        
    # 4) 최종 제거 mask
    mask_remove = mask_dinnershow | mask_multiday | mask_package
    
    df_clean = df[~mask_remove].copy()
    
    print(f"정제 전: {df.shape} 정제 후: {df_clean.shape}")
    return df_clean

In [74]:
df_clean = clean_tickek_data(df)

정제 전: (35630, 41) 정제 후: (35420, 41)


In [76]:
df = df_clean.copy()

In [77]:
# 가격 전처리
import re

# 가격 문자열에서 숫자만 뽑아서 리스트로 변환
def extract_prices_all(x):
    if pd.isna(x):
        return []
    nums = re.findall(r"\d+", str(x).replace(",",""))
    # 1000원 이상만 유효 가격으로 간주
    return [int(n) for n in nums if int(n) >= 1000]

In [78]:
df["티켓가격_list"] = df["티켓가격"].apply(extract_prices_all)
df["최소가격"] = df["티켓가격_list"].apply(lambda x: min(x) if x else None)
df["최대가격"] = df["티켓가격_list"].apply(lambda x: max(x) if x else None)

In [81]:
# 이상치 제거 (최소 가격이 40만원 이상)
mask_min_outlier = df['최소가격'] >= 400_000
df = df[~mask_min_outlier].copy()

In [86]:
df.to_csv("../datasets/KOPIS/performance_cleaned.csv", index=False, encoding="utf-8-sig")